In [ ]:
!pip install -q -U transformers datasets bitsandbytes trl peft huggingface_hub wandb
!pip install typing_extensions==4.5.0
!pip install --upgrade bitsandbytes

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import os
import wandb

os.environ["WANDB_API_KEY"] = ""
wandb.login()

# Configure Weights & Biases to record against our project
os.environ["WANDB_PROJECT"] = "telcollm_runpod"
os.environ["WANDB_LOG_MODEL"] = "checkpoint"
os.environ["WANDB_WATCH"] = "gradients"

In [ ]:
wandb.init(project="telcollm_runpod", name="run_c")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "ibm-granite/granite-3.3-8b-instruct"

config_8bit = BitsAndBytesConfig(load_in_8bit=True)

model_8bit = AutoModelForCausalLM.from_pretrained(


                                                  model_name,
                                                  quantization_config=config_8bit,
                                                  device_map= "auto",
                                                  trust_remote_code =True

                                                  )


In [6]:
model_8bit

GraniteForCausalLM(
  (model): GraniteModel(
    (embed_tokens): Embedding(49159, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x GraniteDecoderLayer(
        (self_attn): GraniteAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GraniteMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=12800, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=12800, bias=False)
          (down_proj): Linear8bitLt(in_features=12800, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): GraniteRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): GraniteRMSNorm((4096,), eps=1e-05)
 

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(tokenizer.padding_side)

tokenizer_config.json:   0%|          | 0.00/9.93k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.48M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/207 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

left


In [8]:
from datasets import load_dataset

dataset = load_dataset("greenwich157/5G-dataset")
dataset

README.md:   0%|          | 0.00/898 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/38.0M [00:00<?, ?B/s]

eval-00000-of-00001.parquet:   0%|          | 0.00/8.13M [00:00<?, ?B/s]

eval_mirror-00000-of-00001.parquet:   0%|          | 0.00/8.04M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50632 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/10126 [00:00<?, ? examples/s]

Generating eval_mirror split:   0%|          | 0/10126 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'chunk_id', 'question', 'answer', 'evaluation_criteria', 'difficulty', 'category', 'model', 'iteration', 'original_question', 'original_answer'],
        num_rows: 50632
    })
    eval: Dataset({
        features: ['document', 'chunk_id', 'question', 'answer', 'evaluation_criteria', 'difficulty', 'category', 'model', 'iteration', 'original_question', 'original_answer'],
        num_rows: 10126
    })
    eval_mirror: Dataset({
        features: ['document', 'chunk_id', 'question', 'answer', 'evaluation_criteria', 'difficulty', 'category', 'model', 'iteration', 'original_question', 'original_answer'],
        num_rows: 10126
    })
})

In [9]:
dataset['train'][0]

{'document': '5G_Faults',
 'chunk_id': 4,
 'question': 'What is the significance of ensuring smooth VoLTE handover from 4G to 5G networks?',
 'answer': 'Ensuring smooth VoLTE handover from 4G to 5G is crucial for maintaining uninterrupted voice services as users move between coverage areas. It requires seamlessly transferring the call session from the 4G infrastructure to the 5G infrastructure without noticeable disruption to the user experience.',
 'evaluation_criteria': '- Correctly identifies the importance of uninterrupted voice service.\\n- Mentions seamless transfer of call sessions between 4G and 5G infrastructure.\\n- Explains the need to avoid disruptions for the user.\\n- Incorrect answers fail to highlight the user experience or the importance of seamless transitions.',
 'difficulty': 3,
 'category': 'factual',
 'model': 'google/gemini-2.0-flash-001',
 'iteration': 1,
 'original_question': 'What is the significance of ensuring smooth VoLTE handover from 4G to 5G networks?',


In [10]:
dataset['eval'][0]

{'document': '5G_Security',
 'chunk_id': 14,
 'question': "Could you detail Javid Taheri's present role and focus of study?",
 'answer': 'Javid Taheri presently holds the role of Associate Professor within the Computer Science Department at Karlstad University, Sweden, and his area of expertise lies in Mobile Computing.',
 'evaluation_criteria': '- Must state that Javid Taheri is an Associate Professor in the Department of Computer Science at Karlstad University, Sweden.\\n- Must mention that his research area is in the field of Mobile Computing.\\n- Incorrect if his position or research area are incorrectly identified.',
 'difficulty': 2,
 'category': 'factual',
 'model': 'google/gemini-2.0-flash-001',
 'iteration': 1,
 'original_question': "What is Javid Taheri's current position and research area?",
 'original_answer': 'Javid Taheri is currently working as an Associate Professor in the Department of Computer Science in Karlstad University, Sweden. He specializes in the field of Mobi

In [12]:
from datasets import DatasetDict

# train_val_split = dataset['train'].train_test_split(test_size=0.2, seed=42)

dataset_run = DatasetDict(

                      {'train':dataset['train'],
                       'eval':dataset['eval']}

)
dataset = dataset_run
dataset

DatasetDict({
    train: Dataset({
        features: ['document', 'chunk_id', 'question', 'answer', 'evaluation_criteria', 'difficulty', 'category', 'model', 'iteration', 'original_question', 'original_answer'],
        num_rows: 50632
    })
    eval: Dataset({
        features: ['document', 'chunk_id', 'question', 'answer', 'evaluation_criteria', 'difficulty', 'category', 'model', 'iteration', 'original_question', 'original_answer'],
        num_rows: 10126
    })
})

In [13]:
def to_chat_template(example):


    messages = [

                {"role": 'system','content':'You are a knowledgeable and helpful mobile telecommunications engineer assistant.'},
                {"role": 'user','content':example['question']},
                {"role": 'assistant','content':example['answer']}

                ]


    return {'text':messages}

dataset = dataset.map(to_chat_template)
dataset



Map:   0%|          | 0/50632 [00:00<?, ? examples/s]

Map:   0%|          | 0/10126 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'chunk_id', 'question', 'answer', 'evaluation_criteria', 'difficulty', 'category', 'model', 'iteration', 'original_question', 'original_answer', 'text'],
        num_rows: 50632
    })
    eval: Dataset({
        features: ['document', 'chunk_id', 'question', 'answer', 'evaluation_criteria', 'difficulty', 'category', 'model', 'iteration', 'original_question', 'original_answer', 'text'],
        num_rows: 10126
    })
})

In [14]:
dataset['train']['text'][0]

[{'content': 'You are a knowledgeable and helpful mobile telecommunications engineer assistant.',
  'role': 'system'},
 {'content': 'What is the significance of ensuring smooth VoLTE handover from 4G to 5G networks?',
  'role': 'user'},
 {'content': 'Ensuring smooth VoLTE handover from 4G to 5G is crucial for maintaining uninterrupted voice services as users move between coverage areas. It requires seamlessly transferring the call session from the 4G infrastructure to the 5G infrastructure without noticeable disruption to the user experience.',
  'role': 'assistant'}]

In [15]:
tokenizer.apply_chat_template(dataset['train']['text'][0], tokenize=False)

'<|start_of_role|>system<|end_of_role|>You are a knowledgeable and helpful mobile telecommunications engineer assistant.<|end_of_text|>\n<|start_of_role|>user<|end_of_role|>What is the significance of ensuring smooth VoLTE handover from 4G to 5G networks?<|end_of_text|>\n<|start_of_role|>assistant<|end_of_role|>Ensuring smooth VoLTE handover from 4G to 5G is crucial for maintaining uninterrupted voice services as users move between coverage areas. It requires seamlessly transferring the call session from the 4G infrastructure to the 5G infrastructure without noticeable disruption to the user experience.<|end_of_text|>\n'

In [16]:
def apply_chat_temp(example):

    new_text = tokenizer.apply_chat_template(example['text'], tokenize=False)

    return {'text':new_text}


dataset = dataset.map(apply_chat_temp)
dataset

Map:   0%|          | 0/50632 [00:00<?, ? examples/s]

Map:   0%|          | 0/10126 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['document', 'chunk_id', 'question', 'answer', 'evaluation_criteria', 'difficulty', 'category', 'model', 'iteration', 'original_question', 'original_answer', 'text'],
        num_rows: 50632
    })
    eval: Dataset({
        features: ['document', 'chunk_id', 'question', 'answer', 'evaluation_criteria', 'difficulty', 'category', 'model', 'iteration', 'original_question', 'original_answer', 'text'],
        num_rows: 10126
    })
})

In [17]:
dataset['train']['text'][0]

'<|start_of_role|>system<|end_of_role|>You are a knowledgeable and helpful mobile telecommunications engineer assistant.<|end_of_text|>\n<|start_of_role|>user<|end_of_role|>What is the significance of ensuring smooth VoLTE handover from 4G to 5G networks?<|end_of_text|>\n<|start_of_role|>assistant<|end_of_role|>Ensuring smooth VoLTE handover from 4G to 5G is crucial for maintaining uninterrupted voice services as users move between coverage areas. It requires seamlessly transferring the call session from the 4G infrastructure to the 5G infrastructure without noticeable disruption to the user experience.<|end_of_text|>\n'

In [18]:
tokenizer(dataset['train']['text'][0])

{'input_ids': [49152, 2946, 49153, 4282, 884, 312, 16969, 444, 461, 17247, 10915, 15559, 508, 31202, 46265, 47330, 32, 0, 203, 49152, 496, 49153, 8197, 438, 322, 41482, 723, 432, 42056, 19052, 30240, 33450, 4091, 1498, 645, 225, 38, 57, 372, 225, 39, 57, 17679, 49, 0, 203, 49152, 17594, 49153, 42098, 4093, 19052, 30240, 33450, 4091, 1498, 645, 225, 38, 57, 372, 225, 39, 57, 438, 5564, 1008, 564, 436, 38812, 707, 1146, 12828, 20309, 6683, 619, 4250, 5169, 3733, 14755, 19492, 32, 2030, 7171, 408, 404, 44996, 1630, 88, 22871, 322, 1476, 3523, 645, 322, 225, 38, 57, 22699, 372, 322, 225, 39, 57, 22699, 2876, 5394, 444, 1214, 26255, 372, 322, 1256, 12409, 32, 0, 203], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [19]:
def tokenize_fn(example):

    return tokenizer(example['text'])


tokenized_dataset = dataset.map(tokenize_fn,batched=True,remove_columns=['question', 'answer', 'evaluation_criteria', 'original_question', 'original_answer', 'document', 'chunk_id', 'difficulty', 'category', 'model', 'iteration'])
tokenized_dataset

Map:   0%|          | 0/50632 [00:00<?, ? examples/s]

Map:   0%|          | 0/10126 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 50632
    })
    eval: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 10126
    })
})

In [20]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False,return_tensors="pt")


In [21]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token

'<|end_of_text|>'

In [ ]:
model_8bit

In [22]:
from peft import  LoraConfig, get_peft_model
# import copy

# model_8bit_clone = copy.deepcopy(model_8bit)


lora_config = LoraConfig(

                         r= 16,
                         lora_alpha = 32,
#                         target_modules = ["q_proj","k_proj","v_proj","o_proj"],
                         target_modules = ["q_proj","k_proj","v_proj","o_proj",
                      "gate_proj","up_proj","down_proj"],
                         lora_dropout = 0.05,
                         bias = "none",
                         task_type = "CAUSAL_LM"

)

model_8bit_lora = get_peft_model(model_8bit,lora_config)
model_8bit_lora.print_trainable_parameters()

trainable params: 49,479,680 || all params: 8,220,344,320 || trainable%: 0.6019


In [ ]:
model_8bit

In [ ]:
model_8bit_lora

In [23]:
from transformers import TrainingArguments
from trl import SFTTrainer


training_args = TrainingArguments(

                                  output_dir = "./results",
                                  per_device_train_batch_size = 16,
                                  per_device_eval_batch_size = 16,
                                  eval_steps = 20,
                                  eval_strategy = "steps",
                                  save_steps = 200,
                                  save_strategy = "steps",
                                  num_train_epochs = 5,
                                  # max_steps = 40,
                                  learning_rate = 1e-4,
                                  weight_decay = 0.01,
                                  # warmup_steps = 100,
                                  lr_scheduler_type = 'linear',
                                  warmup_ratio = 0.03,
                                  bf16 = True,
                                  gradient_accumulation_steps = 16,
                                  optim = "adamw_torch",
                                  logging_steps = 20,
                                  report_to = "wandb"

                                  )

In [24]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 50632
    })
    eval: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 10126
    })
})

In [ ]:
trainer = SFTTrainer(

                     model = model_8bit_lora,
                     # tokenizer = tokenizer,
                     train_dataset =tokenized_dataset['train'],
                     eval_dataset = tokenized_dataset['eval'],
                     args = training_args,
                     data_collator= data_collator,
                     # max_seq_length = 2048,
)

trainer.train()

Truncating train dataset:   0%|          | 0/50632 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/10126 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast

Step,Training Loss,Validation Loss
20,2.693100,2.120220
40,1.440100,1.570918


In [26]:
wandb.finish()

eval/loss,█▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/mean_token_accuracy,▁▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇███████████████████████
eval/num_tokens,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
eval/runtime,▅▂▂▂▃▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▅▅█▄▄▆▅▅▅▅▄▄▄▄▃▄▄▄▄▄
eval/samples_per_second,▄▇▇▇▅██▇▇▇▇▇▇▇▇▇▆▆▆▆▆▄▄▁▅▅▃▄▄▄▅▅▅▅▅▅▅▅▅▅
eval/steps_per_second,▄▇▇▇▅██▇▇▇▇▇▇▇▇▇▆▆▆▆▆▄▄▁▅▅▃▄▄▄▅▅▅▅▆▅▅▅▅▅
train/epoch,▁▁▁▂▂▂▂▂▂▃▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇████
train/grad_norm,█▂▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▅▅▆▆▆▆▆
train/learning_rate,▅████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,█▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁


In [27]:
model_8bit_lora

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GraniteForCausalLM(
      (model): GraniteModel(
        (embed_tokens): Embedding(49159, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-39): 40 x GraniteDecoderLayer(
            (self_attn): GraniteAttention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
 

In [28]:
# uncomment if need to reload session due to out of memory issue
# from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
# model_name = "unsloth/Meta-Llama-3.1-8B-Instruct"
base_model = AutoModelForCausalLM.from_pretrained(


                                                  model_name,

                                                  device_map= "auto",
                                                  trust_remote_code =True

                                                  )

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [29]:
base_model

GraniteForCausalLM(
  (model): GraniteModel(
    (embed_tokens): Embedding(49159, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x GraniteDecoderLayer(
        (self_attn): GraniteAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GraniteMLP(
          (gate_proj): Linear(in_features=4096, out_features=12800, bias=False)
          (up_proj): Linear(in_features=4096, out_features=12800, bias=False)
          (down_proj): Linear(in_features=12800, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): GraniteRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): GraniteRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): GraniteRMSNorm((4

In [30]:
# skip if restart session
lora_path = "./lora_adapters"
model_8bit_lora.save_pretrained(lora_path)

In [31]:
from peft import PeftModel
# lora_path = "./lora_adapters"
base_model =PeftModel.from_pretrained(base_model,lora_path)
base_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GraniteForCausalLM(
      (model): GraniteModel(
        (embed_tokens): Embedding(49159, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-39): 40 x GraniteDecoderLayer(
            (self_attn): GraniteAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
             

In [32]:
base_model = base_model.merge_and_unload()
base_model

GraniteForCausalLM(
  (model): GraniteModel(
    (embed_tokens): Embedding(49159, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x GraniteDecoderLayer(
        (self_attn): GraniteAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GraniteMLP(
          (gate_proj): Linear(in_features=4096, out_features=12800, bias=False)
          (up_proj): Linear(in_features=4096, out_features=12800, bias=False)
          (down_proj): Linear(in_features=12800, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): GraniteRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): GraniteRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): GraniteRMSNorm((4

In [ ]:
# skip if not reloading session
tokenizer = AutoTokenizer.from_pretrained(model_name,padding_side="right",trust_remote_code= True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token

'<|end_of_text|>'

In [34]:
base_model.push_to_hub("greenwich157/granite-3.3-8b-instruct-telcollm-c")
tokenizer.push_to_hub("greenwich157/granite-3.3-8b-instruct-telcollm-c")

model-00004-of-00007.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00007.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00005-of-00007.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00007.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Upload 7 LFS files:   0%|          | 0/7 [00:00<?, ?it/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/2.81G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/greenwich157/granite-3.3-8b-instruct-telcollm-c/commit/fc365b614f09762e8afc72fc041a3d6850869058', commit_message='Upload tokenizer', commit_description='', oid='fc365b614f09762e8afc72fc041a3d6850869058', pr_url=None, repo_url=RepoUrl('https://huggingface.co/greenwich157/granite-3.3-8b-instruct-telcollm-c', endpoint='https://huggingface.co', repo_type='model', repo_id='greenwich157/granite-3.3-8b-instruct-telcollm-c'), pr_revision=None, pr_num=None)